# 1. Importando Dependencias

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pickle

# 2. Funções para detecção dos Keypoints utilizando MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                 
    results = model.process(image)                
    image.flags.writeable = True                  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Desenha conexões da face
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Desenha conexões da pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Desenha conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Desenha conexões da mão direita

In [5]:
def draw_styled_landmarks(image, results):
    # Desenha conexões da face
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Desenha conexões da pose
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Desenha conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Desenha conexões da mão direita
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 3. Atribuindo configurações para os dados

In [7]:
config = dict()

# Path para exportar dataset (onde ficarão os numpy arrays referente aos dados criados)
config['DATA_PATH'] = os.path.join('MP_Data') 

# Ações/Gestos que criaremos (labels dos dados)
actions = ['oi', 'eu_amo_voce']
config['actions'] = np.array(actions)

# Número de dados de cada classe 
config['no_sequences'] = 30

# Quantidade de frames por dado
config['sequence_length'] = 30

In [8]:
with open('config.pickle', 'wb') as handle:
    pickle.dump(config, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# criando nossas pastas
for action in config['actions']: 
    for sequence in range(config['no_sequences']):
        path = f"{config['DATA_PATH']}/{action}/{sequence}"
        try: 
            os.makedirs(path)
        except:
            pass

# 4. Coletando Keypoints para treino e teste utilizando a câmera (criando dataset)

In [10]:
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop nas ações (gestos)
    for action in config['actions']:
        # Loop nos vídeos (qtd de dados de cada ação)
        for sequence in range(config['no_sequences']):
            # Loop nos frames dos vídeos
            for frame_num in range(config['sequence_length']):

                # Leitura da camera
                ret, frame = cap.read()

                # Faz as conexões
                image, results = mediapipe_detection(frame, holistic)
                
                # desenha landmarks
                draw_styled_landmarks(image, results)
                
                # Captura o dado
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                
                # Exporta keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(config['DATA_PATH'], action, str(sequence), str(frame_num))
                print(npy_path)
                np.save(npy_path, keypoints)

                # aperte q para sair do loop
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

MP_Data\oi\0\0
MP_Data\oi\0\1
MP_Data\oi\0\2
MP_Data\oi\0\3
MP_Data\oi\0\4
MP_Data\oi\0\5
MP_Data\oi\0\6
MP_Data\oi\0\7
MP_Data\oi\0\8
MP_Data\oi\0\9
MP_Data\oi\0\10
MP_Data\oi\0\11
MP_Data\oi\0\12
MP_Data\oi\0\13
MP_Data\oi\0\14
MP_Data\oi\0\15
MP_Data\oi\0\16
MP_Data\oi\0\17
MP_Data\oi\0\18
MP_Data\oi\0\19
MP_Data\oi\0\20
MP_Data\oi\0\21
MP_Data\oi\0\22
MP_Data\oi\0\23
MP_Data\oi\0\24
MP_Data\oi\0\25
MP_Data\oi\0\26
MP_Data\oi\0\27
MP_Data\oi\0\28
MP_Data\oi\0\29
MP_Data\oi\1\0
MP_Data\oi\1\1
MP_Data\oi\1\2
MP_Data\oi\1\3
MP_Data\oi\1\4
MP_Data\oi\1\5
MP_Data\oi\1\6
MP_Data\oi\1\7
MP_Data\oi\1\8
MP_Data\oi\1\9
MP_Data\oi\1\10
MP_Data\oi\1\11
MP_Data\oi\1\12
MP_Data\oi\1\13
MP_Data\oi\1\14
MP_Data\oi\1\15
MP_Data\oi\1\16
MP_Data\oi\1\17
MP_Data\oi\1\18
MP_Data\oi\1\19
MP_Data\oi\1\20
MP_Data\oi\1\21
MP_Data\oi\1\22
MP_Data\oi\1\23
MP_Data\oi\1\24
MP_Data\oi\1\25
MP_Data\oi\1\26
MP_Data\oi\1\27
MP_Data\oi\1\28
MP_Data\oi\1\29
MP_Data\oi\2\0
MP_Data\oi\2\1
MP_Data\oi\2\2
MP_Data\oi\2\3


In [ ]:
cap.release()
cv2.destroyAllWindows()